In [1]:
# import tarfile 
# datasource = tarfile.open('liputan6_data.tar.gz') 
# datasource.extractall('./dataset') 
# datasource.close() 

In [2]:
import os
print("canonical/train :",f"{len(os.listdir('./dataset/liputan6_data/canonical/train')):,.0f}")
print("canonical/dev :",f"{len(os.listdir('./dataset/liputan6_data/canonical/dev')):,.0f}")
print("canonical/test :",f"{len(os.listdir('./dataset/liputan6_data/canonical/test')):,.0f}")

canonical/train : 193,883
canonical/dev : 10,972
canonical/test : 10,972


In [3]:
import os
print("xtreme/dev :",f"{len(os.listdir('./dataset/liputan6_data/xtreme/dev')):,.0f}")
print("xtreme/test :",f"{len(os.listdir('./dataset/liputan6_data/xtreme/test')):,.0f}")

xtreme/dev : 4,948
xtreme/test : 3,862


In [39]:
import json
import random 
import pandas as pd

n = random.choices(os.listdir('./dataset/liputan6_data/canonical/dev'),k=1)[0]
n = '2.json'

n = os.listdir('./dataset/liputan6_data/canonical/train')[1]
print(n)


file = open(f'dataset/liputan6_data/canonical/train/{n}','r')
mydict = json.load(file)
print(mydict.keys())
[' '.join(sent).replace(' .','.').replace(' ,',',').replace('( ','(').replace(' )',')').replace('. ','.') for sent in mydict['clean_article']]

100002.json
dict_keys(['id', 'url', 'clean_article', 'clean_summary', 'extractive_summary'])


['Liputan6.com, Jakarta : Perdana Menteri Jepang Junichiro Koizumi meminta maaf atas kekejaman tentara Jepang pada masa Perang Dunia II di Asia.',
 'Permohonan maaf secara formal itu Koizumi utarakan di depan pemimpin negara-negara Asia dan Afrika dalam Konferensi Tingkat Tinggi Asia-Afrika di Jakarta Convention Center, Jumat (22/4).',
 'Koizumi mengatakan, pada masa silam Jepang terlalu ambisius untuk berkuasa.',
 'Ternyata, ambisi itu justru menimbulkan kerusakan dan penderitaan luar biasa bagi penduduk di sejumlah negara, terutama di Asia.',
 '" Untuk itu Jepang minta maaf, " kata Koizumi.',
 'Selain meminta maaf, Koizumi berharap KAA dapat membantu mempererat persahabatan antara Jepang dan negara-negara di Asia serta Afrika.',
 'Jepang, tambah Koizumi, siap memberikan sumbangan dan bantuan bagi negara-negara Asia dan Afrika yang membutuhkan.',
 'Pernyataan Koizumi membuat heran para pengamat.',
 'Menurut mereka, ini kejadian langka.',
 'Mereka menduga, pernyataan Koizumi terkait de

In [65]:
import json 

def get_article(json_filename_path,tobe_get_key='clean_article'):
    file = open(json_filename_path,'r')
    article_dict = json.load(file)
    file.close()
    return article_dict[tobe_get_key]

def cleaning_article_sentence(list_sentence):
    list_sentence_clean = [' '.join(sent).replace(' .','.').replace(' ,',',').replace('( ','(').replace(' )',')').replace('. ','.') for sent in list_sentence]
    # remove "Liputan6.com" intro template
    list_sentence_clean[0] = list_sentence_clean[0].split(':')[-1].strip()
    # remove "( xxxx )" outro template
    list_sentence_clean[-1] = list_sentence_clean[-1].split('.(')[0].strip()+'.'
    pattern = '\[.*\]'
    list_sentence_clean[-1] = re.sub(pattern,'',list_sentence_clean[-1]).replace(' .','.')
    return list_sentence_clean

def data_preparation(json_filename_path,tobe_get_key='clean_article'):
    if tobe_get_key=='clean_article':
        list_sentence = get_article(json_filename_path,tobe_get_key=tobe_get_key)
        list_sentence = cleaning_article_sentence(list_sentence)
        text = ' '.join(list_sentence)
        return text
    elif tobe_get_key=='clean_summary':
        list_sentence = get_article(json_filename_path,tobe_get_key=tobe_get_key)
        # print(list_sentence)
        text = ' '.join([item for row in list_sentence for item in row])
        text = text.replace(' .','.').replace(' ,',',')
        return text

In [69]:
path = 'dataset/liputan6_data/canonical/train'
filenames_path = list(map(lambda x:path+'/'+x, os.listdir(path)))

# articles = list(map(data_preparation,filenames_path))

In [70]:
import timeit
import concurrent.futures
import threading

# Define the code block to be timed
start_time = timeit.default_timer()

# Create a semaphore to limit the number of parallel tasks
semaphore = threading.BoundedSemaphore(5)

# Iterate over the articles
# for filename in filenames_path:
    # Divide the item into chunks of size 700
    
    # Initialize a thread pool executor
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Submit each chunk for summarization
    future_articles = [executor.submit(data_preparation, filename, tobe_get_key='clean_article') for filename in filenames_path]
    # Wait for all the tasks to complete and retrieve the results
    articles = [future.result() for future in concurrent.futures.as_completed(future_articles)]
    
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_summaries = [executor.submit(data_preparation, filename, tobe_get_key='clean_summary') for filename in filenames_path]
    summaries_ref = [future.result() for future in concurrent.futures.as_completed(future_summaries)]

# Print the final summary for the item
# final_summary = ' '.join(summaries)
# print(final_summary)
df_articles = pd.DataFrame(zip(articles,summaries_ref),columns=['articles','summary_ref'])

end_time = timeit.default_timer()


print(f"Execution time: {end_time - start_time:.2f} seconds")

Execution time: 86.98 seconds


In [72]:
df_articles.head()

,articles,summary_ref
0,Jenazah Inspektur Satu Polisi Sugeng dimakamka...,"Anggota DPRD Kota Makassar, Sulsel, dianggap t..."
1,"Tim search and rescue, Selasa (26/4), melanjut...","Penyegelan Kantor KPUD Padang Pariaman, Sumbar..."
2,"Sussongko Suhardjo, tersangka kasus dugaan pen...",Beberapa warga Mamasa meminta digelar jajak pe...
3,Kota Medan dan beberapa wilayah di pesisir tim...,Subsidi BBM setiap bulan dilakukan untuk memba...
4,Ratusan warga Madura yang menjadi korban kerus...,Golok ini memiliki bobot dua ton dan panjang l...


In [73]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193883 entries, 0 to 193882
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   articles     193883 non-null  object
 1   summary_ref  193883 non-null  object
dtypes: object(2)
memory usage: 3.0+ MB


In [76]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("indolem/indobert-base-uncased")
model = AutoModel.from_pretrained("indolem/indobert-base-uncased")

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

c:\Users\bakmi\anaconda3\envs\modeling\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\bakmi\.cache\huggingface\hub\models--indolem--indobert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/234k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

ImportError: 
AutoModel requires the PyTorch library but it was not found in your environment. Checkout the instructions on the
installation page: https://pytorch.org/get-started/locally/ and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [75]:
!pip install transformers

     ---------------------------------------- 0.0/43.6 kB ? eta -:--:--
     ---------------------------------------- 43.6/43.6 kB 2.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ---------------------------------------- 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   - -------------------------------------- 0.2/9.3 MB 7.3 MB/s eta 0:00:02
   -- ------------------------------------- 0.6/9.3 MB 9.7 MB/s eta 0:00:01
   --- ------------------------------------ 0.7/9.3 MB 6.6 MB/s eta 0:00:02
   ---- ----------------------------------- 1.0/9.3 MB 5.7 MB/s eta 0:00:02
   ---- ----------------------------------- 1.1/9.3 MB 5.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.3/9.3 MB 4.9 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.3 MB 4.6 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.3 MB 4.5 MB/s eta 0:00:02
   ------- --------